In [1]:
# !!! This section explains why we should not compare two sentences with these methods. !!! 
# It is not efficient to compare two sentences in Turkish.
# Word count based cosine similarity

import numpy as np

def sentence_to_word_dict(sentence):
    """
    This function takes a sentence as input and returns a dictionary with words as keys and their counts as values.
    
    Args:
        sentence: A string.
    
    Returns:
        A dictionary.
    """
    words = sentence.split()
    word_dict = {}
    for word in words:
        if word in word_dict:
            word_dict[word] += 1
        else:
            word_dict[word] = 1
    return word_dict

sentence_1 = "C'nin dikkat ve özen yükümlülüğüne aykırı davranmış olması nedeniyle kusurlu olduğu değerlendirilemez."
sentence_2 = "C'nin dikkat ve özen yükümlülüğüne aykırı davranmış olması nedeniyle kusurlu olduğu değerlendirilebilir."

dict_1 = sentence_to_word_dict(sentence_1)
dict_2 = sentence_to_word_dict(sentence_2)

word_space = np.unique(list(dict_1.keys()) + list(dict_2.keys()))

# One-hot encoding
binary_vector_1 = [1 if word in dict_1 else 0 for word in word_space]
binary_vector_2 = [1 if word in dict_2 else 0 for word in word_space]

print(binary_vector_1)
print(binary_vector_2)

cosine_similarity = np.dot(binary_vector_1, binary_vector_2) / (np.linalg.norm(binary_vector_1) * np.linalg.norm(binary_vector_2))
print("Cosine similarity (%) :", cosine_similarity * 100)


# TF-IDF based cosine similarity
# Not an efficient way
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

texts = [sentence_1,sentence_2]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
print(similarity)


# Levenshtein distance
def levenshtein_distance(s1, s2):
    len_s1, len_s2 = len(s1) + 1, len(s2) + 1
    dp = np.zeros((len_s1, len_s2))
    for i in range(len_s1):
        dp[i][0] = i
    for j in range(len_s2):
        dp[0][j] = j

    for i in range(1, len_s1):
        for j in range(1, len_s2):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            dp[i][j] = min(dp[i-1][j] + 1, dp[i][j-1] + 1, dp[i-1][j-1] + cost)

    return dp[-1][-1]


print(levenshtein_distance(sentence_1, sentence_2))

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Cosine similarity (%) : 91.66666666666669
0.8477624970048978
5.0


In [4]:
# Load the model Turkish word2vec model
from gensim.models import Word2Vec
from tabulate import tabulate

model = Word2Vec.load("utils/word2vec/w2v_.model")
print(tabulate(model.wv.most_similar("mahkeme"), headers=["Kelime", "Benzerlik Skoru"]))
print("\nWord Vector:", model.wv.get_vector("umut"))

Kelime        Benzerlik Skoru
----------  -----------------
Mahkeme              0.860295
mahkemenin           0.813442
davanın              0.806494
tutuklama            0.799902
soruşturma           0.791518
temyiz               0.771838
mahkemede            0.771153
dava                 0.770335
yargılama            0.769724
savcılık             0.730116

Word Vector: [ 0.1495104  -1.4914255  -0.50925356 -0.9685314   2.1551907   0.10626572
  0.4027821   1.0281931   0.41044936 -1.1525857  -0.0205108   1.0924134
 -1.9218051   1.3797586  -0.63527036 -0.38006008 -0.6512365  -0.96633595
  1.1853794   0.7896848  -0.03258616  0.8834496  -1.6903982   0.9449919
  0.6057014   0.59224516 -1.0036951   2.0536163  -2.1637177  -0.65654767
  1.0522053   0.11371119  1.1112392  -0.43076926  0.13155091 -1.1467836
 -0.8198967   1.1959015  -0.5887494  -1.0079744  -0.25314665  0.5018188
 -0.76072204 -0.30214065 -0.13227591  0.6748753   0.7053673   1.5428567
 -0.08245109  0.76109725 -0.6433578  -1.2249595

In [6]:
from sentence_similarity_comperators import SentenceComparator_Word2Vec,\
                                            SentenceComparator_Ollama,\
                                            SentenceComparator_semantic,\
                                            SentenceComparator_bert_cosine,\
                                            SentenceComparator_SBERT,\
                                            SentenceComparator_NLI,\
                                            SentenceComparator_sentiment_analysis

c:\Users\umutc\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
import pandas as pd
import os

def create_log(log_name, additional_info=""):
    log_name = "log/" + log_name
    with open(log_name, 'w') as f:
        f.write(f"Log file created. ({log_name})\nAdditional Info: {additional_info}\n")

def append_to_log(log_name, message):
    log_name = "log/" + log_name
    with open(log_name, 'a') as f:
        f.write("\n" + message)

def create_excel_file(file_name, sheet_name, data):    
    file_name = "log/" + file_name
    # if exist, remove the file
    try:
        os.remove(file_name)
    except OSError:
        pass
    
    df = pd.DataFrame(data)
    df.to_excel(file_name, sheet_name=sheet_name, index=False)

def append_to_excel(file_name, sheet_name, data):
    file_name = "log/" + file_name
    # Data is one row
    excel_df = pd.read_excel(file_name, sheet_name=sheet_name)
    excel_df = pd.concat([excel_df, pd.DataFrame([data])], ignore_index=True)
    excel_df.to_excel(file_name, sheet_name=sheet_name, index=False)
    
def excel_to_df(file_name, sheet_name):
    file_name = "log/" + file_name
    excel_df = pd.read_excel(file_name, sheet_name=sheet_name)
    return excel_df

        

In [9]:
import time

def test_model(model, model_name):
    """
    This function tests the given model with the given name.

    Args:
        model(SentenceComparator): A SentenceComparator object.
        model_name(str): A string.
    """

    # Start the timer
    start = time.time()
    computation_count = 0

    # Define the sentences to compare
    test_sentences = [
        "C'nin dikkat ve özen yükümlülüğüne aykırı davranmış olması nedeniyle kusurlu olduğu değerlendirilebilir.",
        "C'nin dikkat ve özen yükümlülüğüne aykırı davranmış olması nedeniyle kusurlu olduğu değerlendrilemez.",
        "C kişisi marketten alışveriş yapmıştır ve kasada ödeme yapmadan çıkmıştır.",
        "C kişisi kasada ödeme yapmadan marketten çıkmıştır.",
        "C kasaya ödeme yapması gerekirken yapmamıştır.",
        "C markete girdi ve sonra ödeme yapmadan çıktı.",
        "Şahıs aldığı ürünleri parasını ödemeden çıkmıştır.",
        "C kişisi ödeme yapmayı unutarak marketten çıkmıştır.",
        "C kişisi kesin unutkan birisidir ve ödeme yapmayı unutmuştur.",
        "C kişisi hırsızdır ve hırsızlık suçu işlediği için bu durudman şüphe bile edilemez.",
        "C'nin dikkat ve özen yükümlülüğüne aykırı davranmış olması nedeniyle kusurlu olduğu değerlendrilemez.",
        "C kişisi ödeme yapmadı sonra da marketten çıkarken ödemeyi unuttu.",
        "C kişisi kötü bir insan.",
        "Ben C kişisinin kötü birisi olduğunu biliyorum.",
        "C kişisi iyi bir insan değil.",
        "Kötü bir insan olan C kişisi, ödeme yapmayı unuttuğunu iddia etmektedir.",
        "Kusurlu olan C kişisi, ödeme yapmayı unuttuğunu iddia etmektedir.",
        "C kişisi marketten çıkarken ödeme yapmayı unutmuştur.",
        "C kişisi marketten satın aldığı ürünleri kasada ödeme yapmadan çıkarmıştır.",
        "C'nin kasada ödeme yapmadığı, güvenlik kameralarıyla doğrulanmıştır.",
        "C kişisinin kasada ödeme yapmadan çıkması bilinçli bir eylem olarak değerlendirilebilir.",
        "C kasada ödeme yapmadığı için sorumlu tutulmalıdır.",
        "C kişisinin ödeme yapmadığına dair hiçbir kanıt bulunmamaktadır.",
        "Market çalışanları, C'nin ödeme yapmadığını fark etmiştir.",
        "C kişisi ödeme yapmayı unuttuğunu savunmaktadır.",
        "C'nin kasada ödeme yapmadığı iddiası asılsızdır.",
        "C'nin kasada ödeme yapmaması kasıtlı bir davranış olarak değerlendirilemez.",
        "C, dikkat eksikliği nedeniyle ödeme yapmayı unutmuş olabilir.",
        "C kişisi ödeme yapmadan çıkmayı bir hata olarak tanımlamıştır.",
        "C'nin kasada ödeme yapmadığı, güvenlik kayıtlarıyla teyit edilmiştir.",
        "C'nin kasadan ödeme yapmadan ayrılması bilinçli bir davranış olarak nitelendirilebilir.",
        "C, kasada ödeme yapmadığı için sorumluluk almalıdır.",
        "C'nin ödeme yapmadığına dair herhangi bir kanıt yoktur.",
        "Market çalışanları, C’nin kasada ödeme yapmadığını fark etti.",
        "C kişisi, ödeme yapmayı unuttuğunu iddia ediyor.",
        "C'nin kasada ödeme yapmadığı iddiası gerçeği yansıtmamaktadır.",
        "C'nin ödeme yapmaması kasıtlı olarak değerlendirilemez.",
        "C'nin dikkatsizliği yüzünden ödemeyi unutmuş olabileceği düşünülüyor.",
        "C kişisi, ödeme yapmadan ayrılmayı bir hata olarak kabul etmiştir."
    ]


    # Create log and excel files
    create_log(f"{model_name}_log.txt", "Score is calculated in the range of 0-1. Higher score indicates higher similarity.")  
    create_log("model_exec_times.txt", f"This file contains the execution times of the models. Number of test_sentences: {len(test_sentences)}")

    if model_name == "nli_model":
        create_excel_file(f"{model_name}_log.xlsx", "Results", {"Sentence1": [], "Sentence2": [], "label": [], "score": []})
    elif model_name == "sentiment_analysis":
        create_excel_file(f"{model_name}_log.xlsx", "Results", {"Sentence1": [], "Sentence2": [], "sentiment_1": [], "sentiment_2": []})
    else:
        create_excel_file(f"{model_name}_log.xlsx", "Results", {"Sentence1": [], "Sentence2": [], "Result": []})

    # Compare the sentences
    compared_sentence_pairs = []
    for index, sentence in enumerate(test_sentences):
        for index2, sentence2 in enumerate(test_sentences):
            if index != index2 and ( ( sentence, sentence2 ) not in compared_sentence_pairs\
                               and   ( sentence2, sentence ) not in compared_sentence_pairs):

                # Calculate the similarity is a essential function of SentenceComparator classes
                result = model.calculate_similarity(sentence, sentence2)
                
                append_to_log(f"{model_name}_log.txt", f"\nSentence1: {sentence}\nSentence2: {sentence2}\nResult: {result}")
                if model_name == "nli_model":
                    append_to_excel(f"{model_name}_log.xlsx", "Results", {"Sentence1": sentence, "Sentence2": sentence2, "label": result["label"], "score": result["score"]})
                elif model_name == "sentiment_analysis":
                    append_to_excel(f"{model_name}_log.xlsx", "Results", {"Sentence1": sentence, "Sentence2": sentence2, "sentiment_1": result[0], "sentiment_2": result[1]})
                else:
                    append_to_excel(f"{model_name}_log.xlsx", "Results", {"Sentence1": sentence, "Sentence2": sentence2, "Result": result})
                
                computation_count += 1
                compared_sentence_pairs.append((sentence, sentence2))
        #break # Delete this line for nested for :)

    end = time.time()
    append_to_log("model_exec_times.txt", f"{model_name} Avg comparison time: {(end - start) / computation_count} seconds, Total time: {end - start} seconds")
    append_to_log(f"{model_name}_log.txt", f"\nTotal time: {end - start} seconds")
    print(f"Total time: {end - start} seconds") 


In [59]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#nli_model = SentenceComparator_NLI("microsoft/deberta-large-mnli")
#test_model(nli_model, "nli_model")
#
#semantic_similarity = SentenceComparator_semantic("paraphrase-MiniLM-L6-v2")
#test_model( semantic_similarity, "semantic_similarity")
#
#bert_cosine_similarity = SentenceComparator_bert_cosine("bert-base-multilingual-cased")
#test_model(bert_cosine_similarity, "bert_cosine_similarity")
#
#sbert_similarity = SentenceComparator_SBERT("paraphrase-multilingual-mpnet-base-v2")
#test_model(sbert_similarity, "sbert_similarity")
#
#word2vec_sim = SentenceComparator_Word2Vec("utils/word2vec/w2v_.model")
#test_model(word2vec_sim, "word2vec_sim")
#
#sentiment_analysis = SentenceComparator_sentiment_analysis()
#test_model(sentiment_analysis, "sentiment_analysis")
#
#sys_prompt= "Sen bir text-miner algoritmasın.\
#                Cümleleri sadece anlamsal olarak değerlendir.\
#                İstenen dönüş: değerlendirme:<benzer anlam->1, farklı anlam->0>.\
#                Bu formate göre bir dönüş sağla ve sadece anlama odaklan."
#
#ollama_model_llama3 = SentenceComparator_Ollama(
#    llama_version="llama3.1",
#    modelfile_system= sys_prompt,
#    temperature=0.4
#)
#test_model(ollama_model_llama3, "ollama_model_llama3.1")


In [22]:
########################################

# For bert_cos_sim
bert_cos_sim = excel_to_df("bert_cosine_similarity_log.xlsx", "Results")
bert_cos_sim["Result"] = bert_cos_sim["Result"].str.replace(r'[\[\]]','',regex=True).astype(float)

########################################

# For sbert_similarity
sbert_cos_df = excel_to_df("sbert_similarity_log.xlsx", "Results")
sbert_cos_df["Result"] = sbert_cos_df["Result"].str.replace(r'[\[\]()tensor]','',regex=True).astype(float)

########################################

# For nli_model_log.xlsx
nli_df = excel_to_df("nli_model_log.xlsx", "Results")

# Dummy encoding
dummy = pd.get_dummies(nli_df["label"])
nli_df.drop("label", axis=1, inplace=True)
nli_df = pd.concat([nli_df, dummy], axis=1)

nli_df.rename(columns={"score":"Result"}, inplace=True)

########################################

# For semantic_similarity_log.xlsx
semantic_df = excel_to_df("semantic_similarity_log.xlsx", "Results")
semantic_df["Result"] = semantic_df["Result"].str.replace(r'[\[\]()tensor]','',regex=True).astype(float)

########################################

# For word2vec_sim_log.xlsx
word2vec_df = excel_to_df("word2vec_sim_log.xlsx", "Results")

########################################

# Sentiment Analysis
sentiment_df = excel_to_df("sentiment_analysis_log.xlsx", "Results")

"""
There are two options for processing sentiment analysis results.

1. We can calculate sentiment score as a difference between sentiment_1 and sentiment_2. 
If they are equal, the score will be 1. Otherwise, the score will be 0. With this approach, we have a binary classification problem.

2. We can use sentiment_1 and sentiment_2 as two separate features. With this approach, 
we have a multi-class classification problem. The first sentence have two labels, and the second sentence have two labels.
Threfore, we have 4 labels in total.

"""
# Dummy encoding Option 2
dummy_1 = pd.get_dummies(sentiment_df["sentiment_1"])
dummy_2 = pd.get_dummies(sentiment_df["sentiment_2"])

rename_all_columns = lambda df, suffix: df.rename(columns={col: col + suffix for col in df.columns})

dummy_1 = rename_all_columns(dummy_1, "_setnence1")
dummy_2 = rename_all_columns(dummy_2, "_setnence2")

sentiment_df.drop(["sentiment_1", "sentiment_2"], axis=1, inplace=True)
sentiment_df = pd.concat([sentiment_df, dummy_1, dummy_2], axis=1)

########################################

# For ollama_model_llama3.1_log.xlsx
ollama_df = excel_to_df("ollama_model_llama3.1_log.xlsx", "Results")
ollama_df["Result"] = ollama_df["Result"].str.replace(r'[\[\]()tensorDdeğerlendirme:Cüaıbzkfakı .23456789]','',regex=True).astype(int)
ollama_df

# Concatenate all the results
#all_results = pd.concat([bert_cos_sim["Sentence1"],bert_cos_sim["Sentence2"],bert_cos_sim["Result"], sbert_cos_df["Result"], nli_df["Result"], semantic_df["Result"], word2vec_df["Result"], sentiment_df["Result"], ollama_df["Result"]],
#                        axis=1, 
#                        keys=["Sentence1", "Sentence2", "bert_cos_sim", "sbert_cos_sim", "nli_model", "semantic_similarity", "word2vec_sim", "sentiment_analysis", "ollama_model_llama3.1"])


def concat_columns_except_sentences(df_list, df_list_names):
    initial_df = df_list[0].iloc[:,:2]
    for index, df in enumerate(df_list):
        df_except_sentences = df[ df.columns.difference(["Sentence1", "Sentence2"]) ] * 1
        df_except_sentences.columns = [f"{df_list_names[index]}_{col}" for col in df_except_sentences.columns]
        initial_df = pd.concat([initial_df, df_except_sentences], axis=1)
    return initial_df

all_results = concat_columns_except_sentences(
    [bert_cos_sim, sbert_cos_df, nli_df, semantic_df, word2vec_df, sentiment_df, ollama_df], 
    ["bert_cos_sim", "sbert_cos_sim", "nli_model", "semantic_similarity", "word2vec_sim", "sentiment_analysis", "ollama_model_llama3.1"])
             
all_results

,Sentence1,Sentence2,bert_cos_sim_Result,sbert_cos_sim_Result,nli_model_CONTRADICTION,nli_model_ENTAILMENT,nli_model_NEUTRAL,nli_model_Result,semantic_similarity_Result,word2vec_sim_Result,sentiment_analysis_LABEL_1_1,sentiment_analysis_LABEL_1_2,sentiment_analysis_LABEL_2_1,sentiment_analysis_LABEL_2_2,ollama_model_llama3.1_Result
0,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,0.964577,0.9383,0,1,0,0.769148,0.9611,0.946022,1,0,0,1,0
1,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,C kişisi marketten alışveriş yapmıştır ve kasa...,0.636648,0.1996,1,0,0,0.589680,0.7214,0.398693,1,1,0,0,1
2,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,C kişisi kasada ödeme yapmadan marketten çıkmı...,0.625543,0.1646,1,0,0,0.747958,0.7042,0.312992,1,1,0,0,1
3,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,C kasaya ödeme yapması gerekirken yapmamıştır.,0.657169,0.4852,1,0,0,0.710494,0.6675,0.391442,1,0,0,1,1
4,C'nin dikkat ve özen yükümlülüğüne aykırı davr...,C markete girdi ve sonra ödeme yapmadan çıktı.,0.587133,0.2732,1,0,0,0.737624,0.6462,0.396546,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,C'nin kasada ödeme yapmadığı iddiası gerçeği y...,C'nin dikkatsizliği yüzünden ödemeyi unutmuş o...,0.697819,0.7362,1,0,0,0.844054,0.6441,0.515694,0,1,1,0,1
700,C'nin kasada ödeme yapmadığı iddiası gerçeği y...,"C kişisi, ödeme yapmadan ayrılmayı bir hata ol...",0.718892,0.6233,1,0,0,0.514596,0.6645,0.496692,0,1,1,0,1
701,C'nin ödeme yapmaması kasıtlı olarak değerlend...,C'nin dikkatsizliği yüzünden ödemeyi unutmuş o...,0.703241,0.6767,1,0,0,0.601376,0.7230,0.457836,1,1,0,0,0
702,C'nin ödeme yapmaması kasıtlı olarak değerlend...,"C kişisi, ödeme yapmadan ayrılmayı bir hata ol...",0.764747,0.6354,1,0,0,0.867559,0.6490,0.658151,1,1,0,0,0


In [13]:
# Save the results to an excel file
#all_results.to_excel("log/all_results.xlsx", index=False)

# Read the results from the excel file
#all_results = pd.read_excel("log/all_results.xlsx")

# Calculate the correlation matrix
all_results_parameters = all_results.drop(["Sentence1", "Sentence2"], axis=1)


In [14]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# Normalize lib
from sklearn.preprocessing import StandardScaler
# Test train split
from sklearn.model_selection import train_test_split

# Define the features and target
y = all_results_parameters["ollama_model_llama3.1_Result"]
X = all_results_parameters.drop("ollama_model_llama3.1_Result", axis=1)

normalized_X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(normalized_X, y, test_size=0.2, random_state=42)

In [15]:
# Calculate the accuracy
from sklearn.metrics import accuracy_score

# LOGISTIC REGRESSION
# Create the model
log_reg = LogisticRegression()

# Fit the model
log_reg.fit(X_train, y_train)

# Predict the target
y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Logistic Reg, Accuracy: ", accuracy)

Logistic Reg, Accuracy:  0.723404255319149


In [16]:
X

,bert_cos_sim_Result,sbert_cos_sim_Result,nli_model_CONTRADICTION,nli_model_ENTAILMENT,nli_model_NEUTRAL,nli_model_Result,semantic_similarity_Result,word2vec_sim_Result,sentiment_analysis_LABEL_1,sentiment_analysis_LABEL_1,sentiment_analysis_LABEL_2,sentiment_analysis_LABEL_2
0,0.964577,0.9383,0,1,0,0.769148,0.9611,0.946022,1,0,0,1
1,0.636648,0.1996,1,0,0,0.589680,0.7214,0.398693,1,1,0,0
2,0.625543,0.1646,1,0,0,0.747958,0.7042,0.312992,1,1,0,0
3,0.657169,0.4852,1,0,0,0.710494,0.6675,0.391442,1,0,0,1
4,0.587133,0.2732,1,0,0,0.737624,0.6462,0.396546,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.697819,0.7362,1,0,0,0.844054,0.6441,0.515694,0,1,1,0
700,0.718892,0.6233,1,0,0,0.514596,0.6645,0.496692,0,1,1,0
701,0.703241,0.6767,1,0,0,0.601376,0.7230,0.457836,1,1,0,0
702,0.764747,0.6354,1,0,0,0.867559,0.6490,0.658151,1,1,0,0


In [11]:
log_reg.coef_[0]

array([ 0.44644703,  0.37668813,  0.05561013,  0.06242784, -0.09295003,
        0.13803581,  0.3736199 , -0.55249954, -0.07450976, -0.01439088,
        0.07450976,  0.01439088])

In [12]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

def print_conf_matrix(y_test, y_pred):
    conf_matrix = confusion_matrix(y_test, y_pred)

    precision = conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[0][1])
    recall = conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[1][0])

    print("Precision: ", precision)
    print("Recall: ", recall)

    print("True Positive: ", conf_matrix[1][1])
    print("True Negative: ", conf_matrix[0][0])
    print("False Positive: ", conf_matrix[0][1])
    print("False Negative: ", conf_matrix[1][0])
    
print_conf_matrix(y_test, y_pred)

Precision:  0.5384615384615384
Recall:  0.175
True Positive:  7
True Negative:  95
False Positive:  6
False Negative:  33


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score

# Assuming X_train, y_train, X_test, y_test are defined
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#model.fit(X_train, y_train, epochs=100, batch_size=5)


c:\Users\umutc\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
def keras_model_tuner(hp):
    hidden_layer_num = hp.Int('hidden_layer_num', min_value=1, max_value=5, step=1)
    layer_unit = hp.Int('layer_unit', min_value=16, max_value=128, step=16)

    model = models.Sequential()
    model.add(layers.Dense(layer_unit, activation='relu', input_shape=(X_train.shape[1],)))

    for i in range(hidden_layer_num):
        model.add(layers.Dense(layer_unit, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))    

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

from kerastuner import RandomSearch

tuner = RandomSearch(
    keras_model_tuner,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='log',
    project_name='ollama_model2'
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001 )

#tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[reduce_lr])

Reloading Tuner from log\ollama_model2\tuner0.json


In [18]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    keras_model_tuner,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='log',
    project_name='ollama_model2'
)

tuner.reload()
model = tuner.get_best_models(num_models=1)[0]

y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: ", accuracy)

print_conf_matrix(y_test, y_pred)

Reloading Tuner from log\ollama_model2\tuner0.json


c:\Users\umutc\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Accuracy:  0.75177304964539
Precision:  0.8571428571428571
Recall:  0.15
True Positive:  6
True Negative:  100
False Positive:  1
False Negative:  34


In [19]:
def add_output_columns(df, y_test, y_pred):
    """
    This function adds the Confidence, Real Value, and Prediction columns to the dataframe.

    Args:
        df: A pandas dataframe.

    Returns:
        df: A pandas dataframe.
    """
    if type(df) != pd.DataFrame:
        df = pd.DataFrame(df)
    df["Confidence"] = 1.0
    df["Real Value"] = 5
    df["Prediction"] = 5
    df["Accuracy"] = 5
    #print(df.head())
    df.reset_index(drop=True, inplace=True)
    
    for index, val in enumerate(y_pred):
        
        real =  y_test.iloc[index]
        pred = 0 if val < 0.5 else 1
        confidence = (val-0.5)*2 if pred == 1 else (0.5-val)*2
        
        df["Confidence"][index] = confidence
        df["Real Value"][index] = real
        df["Prediction"][index] = pred
        df["Accuracy"][index] = 1 if real == pred else 0

    return df

def calculate_accuracy(x,y,predictor):
    """
    This function calculates the accuracy of the predictor.

    Args:
        x: A pandas dataframe.
        y: A pandas dataframe.
        predictor: A predictor model.

    Returns:
        accuracy: A float.
    """
    y_pred = predictor.predict(x)
    y_pred = [1 if val > 0.5 else 0 for val in y_pred]
    accuracy = accuracy_score(y, y_pred)
    return accuracy

In [20]:
# Test
print("Test Accuracy: ", calculate_accuracy(X_test, y_test, model))
# Train
print("Train Accuracy: ", calculate_accuracy(X_train, y_train, model))
# Entire
print("Entire Accuracy: ", calculate_accuracy(normalized_X, y, model)) 

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Test Accuracy:  0.75177304964539
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Train Accuracy:  0.7122557726465364
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Entire Accuracy:  0.7201704545454546


In [89]:
high_acc

,0,1,2,3,4,5,6,7,8,9,10,11,Confidence,Real Value,Prediction,Accuracy
0,0.815188,1.590520,-0.892143,3.009509,-0.918113,-0.817017,-0.407003,-1.016843,0.570789,0.377964,-0.570789,-0.377964,0.017966,1,0,0
1,1.271673,0.764906,1.120897,-0.332280,-0.918113,-0.325933,0.588475,1.624017,0.570789,-2.645751,-0.570789,2.645751,0.436070,1,0,0
2,0.787798,-0.582501,-0.892143,3.009509,-0.918113,-1.225806,0.475152,-0.524713,0.570789,0.377964,-0.570789,-0.377964,0.215165,1,0,0
3,0.678161,-0.317145,1.120897,-0.332280,-0.918113,0.638371,0.522652,-0.037228,0.570789,0.377964,-0.570789,-0.377964,0.401310,1,0,0
4,0.688566,-0.634903,1.120897,-0.332280,-0.918113,-0.574057,-1.054368,0.070940,0.570789,0.377964,-0.570789,-0.377964,0.518599,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.286441,-0.745840,1.120897,-0.332280,-0.918113,-0.842474,0.195577,1.072469,0.570789,0.377964,-0.570789,-0.377964,0.561216,0,0,1
137,0.351238,-0.550725,-0.892143,-0.332280,1.089190,-0.255695,0.718084,-0.655045,0.570789,0.377964,-0.570789,-0.377964,0.351091,0,0,1
138,0.256504,0.109878,1.120897,-0.332280,-0.918113,1.441526,0.229506,-0.615179,-1.751960,0.377964,1.751960,-0.377964,0.221825,0,0,1
139,-0.433858,0.238654,-0.892143,-0.332280,1.089190,1.138172,0.382865,-0.462409,0.570789,0.377964,-0.570789,-0.377964,0.346781,1,0,0


In [92]:
# TEST DATASET

high_accuracy_limit = 0.6

keras_predict_df_test = pd.DataFrame(model.predict(X_test), columns=["Prediction"])

result_df_test = add_output_columns(X_test, y_test, keras_predict_df_test["Prediction"])

faults_test = result_df_test[result_df_test["Accuracy"] == 0]

high_acc = result_df_test[result_df_test["Confidence"] > high_accuracy_limit]["Accuracy"].value_counts()
print("Error rate: %", 100* high_acc[0] / (high_acc[0] + high_acc[1]), "      Number of high confidence predictions: ", high_acc[0] + high_acc[1])

print("Number of faults: ", faults_test.shape[0], "   Faults from high confidence predictions: ", faults_test[ faults_test["Confidence"] > high_accuracy_limit ].shape[0])
faults_test

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Error rate: % 3.225806451612903       Number of high confidence predictions:  31
Number of faults:  35    Faults from high confidence predictions:  1


,0,1,2,3,4,5,6,7,8,9,10,11,Confidence,Real Value,Prediction,Accuracy
0,0.815188,1.590520,-0.892143,3.009509,-0.918113,-0.817017,-0.407003,-1.016843,0.570789,0.377964,-0.570789,-0.377964,0.017966,1,0,0
1,1.271673,0.764906,1.120897,-0.332280,-0.918113,-0.325933,0.588475,1.624017,0.570789,-2.645751,-0.570789,2.645751,0.436070,1,0,0
2,0.787798,-0.582501,-0.892143,3.009509,-0.918113,-1.225806,0.475152,-0.524713,0.570789,0.377964,-0.570789,-0.377964,0.215165,1,0,0
3,0.678161,-0.317145,1.120897,-0.332280,-0.918113,0.638371,0.522652,-0.037228,0.570789,0.377964,-0.570789,-0.377964,0.401310,1,0,0
4,0.688566,-0.634903,1.120897,-0.332280,-0.918113,-0.574057,-1.054368,0.070940,0.570789,0.377964,-0.570789,-0.377964,0.518599,1,0,0
14,-0.880661,-0.038967,1.120897,-0.332280,-0.918113,-0.522799,0.362507,-0.760867,0.570789,0.377964,-0.570789,-0.377964,0.316467,1,0,0
16,0.277258,-0.140984,-0.892143,-0.332280,1.089190,1.540841,0.336721,0.105899,0.570789,0.377964,-0.570789,-0.377964,0.372742,1,0,0
18,-1.039912,0.067510,-0.892143,-0.332280,1.089190,0.510728,0.276328,-0.208648,0.570789,0.377964,-0.570789,-0.377964,0.470710,1,0,0
25,-1.168701,-0.763679,-0.892143,-0.332280,1.089190,-1.038817,-0.791758,-0.845175,0.570789,0.377964,-0.570789,-0.377964,0.654165,1,0,0
33,-0.712509,-0.548495,1.120897,-0.332280,-0.918113,-0.409182,-0.968867,-1.415428,0.570789,0.377964,-0.570789,-0.377964,0.373990,1,0,0


In [93]:
print_conf_matrix(result_df_test["Real Value"], result_df_test["Prediction"])

Precision:  0.8571428571428571
Recall:  0.15
True Positive:  6
True Negative:  100
False Positive:  1
False Negative:  34


In [94]:
# ENTIRE DATASET
# Predicts
keras_predict_df = pd.DataFrame(model.predict(normalized_X), columns=["Prediction"])

#sort the dataframe by ID
result_df = add_output_columns( 
    df = normalized_X,
    y_test = y,
    y_pred = keras_predict_df["Prediction"]
)
faults = result_df[ result_df["Accuracy"] == 0 ]

# Accuracy counts of the model where the confidence is greater than high_accuracy_limit
high_acc = result_df[result_df["Confidence"] > high_accuracy_limit]["Accuracy"].value_counts()
print("Error rate: %", 100* high_acc[0] / (high_acc[0] + high_acc[1]), "      Number of high confidence predictions: ", high_acc[0] + high_acc[1])

print("Number of faults: ", faults.shape[0], "   Faults from high confidence predictions: ", faults[ faults["Confidence"] > high_accuracy_limit ].shape[0])
faults

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Error rate: % 5.384615384615385       Number of high confidence predictions:  130
Number of faults:  197    Faults from high confidence predictions:  7


,0,1,2,3,4,5,6,7,8,9,10,11,Confidence,Real Value,Prediction,Accuracy
0,2.897630,2.112313,-0.892143,3.009509,-0.918113,0.880965,1.959885,3.242402,0.570789,-2.645751,-0.570789,2.645751,0.122103,0,1,0
1,-0.976951,-2.005724,1.120897,-0.332280,-0.918113,-0.307061,0.333328,-0.925751,0.570789,0.377964,-0.570789,-0.377964,0.441499,1,0,0
2,-1.108159,-2.200838,1.120897,-0.332280,-0.918113,0.740699,0.216613,-1.578397,0.570789,0.377964,-0.570789,-0.377964,0.459541,1,0,0
3,-0.734488,-0.413587,1.120897,-0.332280,-0.918113,0.492693,-0.032426,-0.980969,0.570789,-2.645751,-0.570789,2.645751,0.458125,1,0,0
5,-1.355766,-1.871931,1.120897,-0.332280,-0.918113,-0.877772,0.424258,-1.659455,0.570789,0.377964,-0.570789,-0.377964,0.443167,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,-0.534201,0.104303,1.120897,-0.332280,-0.918113,-1.056210,-1.297979,-1.242834,0.570789,0.377964,-0.570789,-0.377964,0.366868,1,0,0
696,0.331730,1.740480,-0.892143,3.009509,-0.918113,-1.424415,-0.217000,-1.076460,0.570789,0.377964,-0.570789,-0.377964,0.040395,1,0,0
699,-0.254197,0.985665,1.120897,-0.332280,-0.918113,1.376823,-0.191214,-0.034734,-1.751960,0.377964,1.751960,-0.377964,0.278953,1,0,0
700,-0.005210,0.356280,1.120897,-0.332280,-0.918113,-0.804090,-0.052784,-0.179440,-1.751960,0.377964,1.751960,-0.377964,0.144541,1,0,0


In [49]:
result_df.describe()

,0,1,2,3,4,5,Confidence,Real Value,Prediction,Accuracy
count,7.040000e+02,7.040000e+02,7.040000e+02,7.040000e+02,7.040000e+02,7.040000e+02,704.000000,704.000000,704.000000,704.000000
mean,8.124814e-16,1.009294e-16,7.065056e-17,2.573699e-16,-5.450186e-16,-3.027881e-17,0.703830,0.301136,0.272727,0.840909
std,1.000711e+00,1.000711e+00,1.000711e+00,1.000711e+00,1.000711e+00,1.000711e+00,0.308343,0.459078,0.445678,0.366021
min,-3.375819e+00,-2.617827e+00,-1.976928e+00,-3.428721e+00,-2.103059e+00,-1.498079e+00,0.001044,0.000000,0.000000,0.000000
25%,-6.171947e-01,-6.411746e-01,-8.397758e-01,-5.196470e-01,-7.021904e-01,-1.498079e+00,0.487678,0.000000,0.000000,1.000000
50%,8.630704e-02,-1.583170e-02,-1.698798e-02,2.159341e-01,-1.723928e-01,6.675217e-01,0.822868,0.000000,0.000000,1.000000
75%,6.399473e-01,7.538961e-01,8.097366e-01,6.919582e-01,6.037603e-01,6.675217e-01,0.979437,1.000000,1.000000,1.000000
max,3.316165e+00,2.456273e+00,2.317426e+00,2.223853e+00,3.653464e+00,6.675217e-01,1.000000,1.000000,1.000000,1.000000


In [50]:
high_confidence = result_df[ result_df["Confidence"] > high_accuracy_limit ]

high_confidence["Accuracy"].value_counts()

Accuracy
1    353
0     17
Name: count, dtype: int64